In [1]:
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch as T
import numpy as np
import pytorch_lightning as pl
import pandas as pd
import pickle
from functools import reduce
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm

In [2]:
train_data = pd.read_csv("../data/type2_train.csv")
x_train, y_train = train_data.drop("fraud_ind", 1), train_data["fraud_ind"]

In [3]:
test_data = pd.read_csv("../data/type2_test.csv")
x_test, y_test = test_data.drop("fraud_ind", 1), test_data['fraud_ind']

In [4]:
x_train

,acqic,bacno,cano,conam,iterm,locdt,loctm,mcc,mchno,scity,...,stocn_98,stocn_99,stocn_100,stocn_101,stocn_102,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4
0,0.999564,0.691103,0.178303,0.071274,0.0,0.359551,0.731703,0.995643,0.574337,0.000000,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.999564,0.094012,0.882785,0.071274,0.0,0.056180,0.646121,0.995643,0.574337,0.000000,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.975596,0.958965,0.140470,0.140955,0.0,0.044944,0.732949,0.538126,0.488215,0.491830,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.867955,0.646705,0.381116,0.098999,0.0,0.056180,0.771867,0.572985,0.907731,0.871983,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.931290,0.576228,0.230713,0.250596,0.0,0.056180,0.731585,0.738562,0.000000,0.879179,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141335,0.876235,0.015310,0.253565,0.000191,0.0,0.325843,0.980827,0.540305,0.757906,0.871983,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1141336,0.261621,0.078294,0.643971,0.157869,0.0,0.000000,0.423841,0.437908,0.001917,0.277620,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1141337,0.918361,0.555317,0.071198,0.080233,0.0,0.033708,0.812183,0.455338,0.369985,0.871983,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1141338,0.468623,0.885424,0.544930,0.060388,0.0,0.134831,0.433711,0.418301,0.872497,0.218558,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [5]:
x_test

,acqic,bacno,cano,conam,iterm,locdt,loctm,mcc,mchno,scity,...,stocn_98,stocn_99,stocn_100,stocn_101,stocn_102,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4
0,0.000000,0.820750,0.214335,0.064591,0.0,0.089888,0.445476,0.982571,0.000000,0.871983,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.000000,0.478244,0.010758,0.064591,0.0,0.056180,0.444645,0.982571,0.000000,0.871983,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.000000,0.921712,0.926955,0.064591,0.0,0.067416,0.444641,0.982571,0.000000,0.871983,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.975596,0.920693,0.021286,0.129053,0.0,0.078652,0.601918,0.716776,0.363577,0.872433,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.999709,0.673377,0.625451,0.000000,0.0,0.089888,0.608822,1.000000,0.023822,0.000000,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380442,0.811737,0.581195,0.525191,0.000191,0.0,0.179775,0.823168,0.644880,0.716292,0.513566,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
380443,0.902818,0.523361,0.794777,0.084510,0.0,0.146067,0.567501,0.629630,0.530729,0.518663,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
380444,0.905142,0.852602,0.787624,0.079675,0.0,0.303371,0.658746,0.533769,0.178158,0.871983,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
380445,0.876235,0.277057,0.925591,0.000191,0.0,0.258427,0.912099,0.812636,0.767092,0.871983,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [20]:
y_train.value_counts()/y_train.count()

0    0.98661
1    0.01339
Name: fraud_ind, dtype: float64

In [21]:
y_test.value_counts()/y_test.count()

0    0.986668
1    0.013332
Name: fraud_ind, dtype: float64

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, labels):
        self.data = df.values.astype(np.float32)
        self.labels = labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return T.tensor(self.data[idx]), self.labels[idx]

# Vanilla KNN

In [6]:
def euclidean_distances(x_train, x_test):
    return T.linalg.norm(x_train - x_test.unsqueeze(1), dim=2)

x_train_tensor = T.tensor(x_train.values).cuda()
y_train_tensor = T.tensor(y_train.values).cuda()

def KNN(x_test):
    distance = euclidean_distances(x_train_tensor, x_test)
    top_k_xvals, top_k_indices = T.topk(T.neg(distance), 3)
    prediction_indices = y_train_tensor[top_k_indices]
    prediction = T.mode(prediction_indices)
    return prediction.values

val_dataloader = DataLoader(MyDataset(x_test, T.tensor(y_test.values)), batch_size=4, shuffle=False, num_workers=16)
preds = []
labels = []
for x, y in tqdm(val_dataloader):
    preds += KNN(x.cuda()).cpu().detach().tolist()
    labels += y.tolist()

  0%|          | 0/95112 [00:00<?, ?it/s]

In [7]:
((TN, FP), (FN, TP)) = metrics.confusion_matrix(labels, preds, labels=[0, 1])
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f_score = 2 * (precision * recall) / (precision + recall)
((TN, FP), (FN, TP)), (precision, recall, f_score)

(((374147, 1228), (2206, 2866)),
 (0.7000488519785051, 0.5650630914826499, 0.6253545712415448))

# Autoencoder & KNN

In [46]:
train_batch_size = 2048
val_batch_size = 64
num_train_epochs = 50

In [81]:
class Model(pl.LightningModule):
    def __init__(self, dim, x_train, y_train, x_val, y_val, K=3):
        super(Model, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(dim, 31),
            nn.Tanh(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(31, dim),
            nn.Sigmoid(),
        )
        self.K = K
        self.x_train = x_train
        self.y_train = T.tensor(y_train.values)
        self.x_val = x_val
        self.y_val = T.tensor(y_val.values)
        self.x_train_embedded = None

    def forward(self, x):
        z = self.encoder(x)
        z = self.decoder(z)
        return z
    
    def train_dataloader(self):
        return DataLoader(MyDataset(self.x_train, self.y_train), batch_size=train_batch_size, shuffle=False, pin_memory=True, num_workers=8)
    
    def training_step(self, batch, batch_idx):
        data, labels = batch
        loss = F.mse_loss(self(data), data)
        return loss
    
    def on_train_end(self):
        self._update_embedding()
    
    def _update_embedding(self):
        x_train_embedded = []
        for batch, labels in self.train_dataloader():
            x_train_embedded.append(self.encoder(batch.cuda()))
        self.x_train_embedded = T.cat(x_train_embedded)
    
    def _euclidean_distances(self, x_train, x_test):
        return T.linalg.norm(x_train - x_test.unsqueeze(1), dim=2)

    def _KNN(self, x_test, K):
        distance = self._euclidean_distances(self.x_train_embedded, x_test)
        top_k_xvals, top_k_indices = T.topk(T.neg(distance), K)
        prediction_indices = self.y_train[top_k_indices]
        prediction = T.mode(prediction_indices)
        return prediction.values

    def configure_optimizers(self):
        optimizer = T.optim.Adam(self.parameters(), lr=5e-3)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': T.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[x for x in range(10, num_train_epochs, 20)], gamma=0.1),
                'interval': 'epoch'
            }
        }

In [22]:
model = Model(x_train.shape[1], x_train, y_train, x_test, y_test, K=3)

trainer = pl.Trainer(
    gpus=1,
    max_epochs=num_train_epochs,
)
trainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 29.7 K
1 | decoder | Sequential | 29.8 K
---------------------------------------
59.5 K    Trainable params
0         Non-trainable params
59.5 K    Total params
0.238     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

1

In [ ]:
trainer.save_checkpoint("autoencoder_type2_128.ckpt")

In [ ]:
def load_model(path, K):
    model = Model.load_from_checkpoint(path, dim=x_train.shape[1], x_train=x_train, y_train=y_train, x_val=x_test, y_val=y_test, K=K)
    model = model.cuda()
    model._update_embedding()
    return model

# Testing

In [88]:
test_dataloader = DataLoader(MyDataset(x_test, T.tensor(y_test.values)), batch_size=30, shuffle=False, pin_memory=True, num_workers=16)

In [91]:
from tqdm.notebook import tqdm

preds = []
labels = []

model = model.cuda()
for x, y in tqdm(test_dataloader):
    x = x.cuda()
    preds += model._KNN(model.encoder(x), 3)
    labels += y

  0%|          | 0/12682 [00:00<?, ?it/s]

In [92]:
((TN, FP), (FN, TP)) = metrics.confusion_matrix(labels, preds, labels=[0, 1])

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f_score = 2 * (precision * recall) / (precision + recall)
print(f"precision: {precision}\nrecall: {recall}\nf_score: {f_score}")

precision: 0.7028172525554724
recall: 0.5557965299684543
f_score: 0.6207200264229881
